# Python Problem: python script which can fetch all the tweets done by ​midas@IIITD​ twitter handle and dump the responses into JSONlines file.

#### Importing the libraries
 
 - json: for saving and loading of data
 - requests: for sending GET request to Twitter API
 - requests_oauthlib: for authentication on Twitter API
 - BeautifulSoup for webScrapping
 - if needed_ tweepy to extract the tweets 

In [7]:
import sys
import json
import requests
from requests_oauthlib import OAuth1
from bs4 import BeautifulSoup
import tweepy

#### Function to authenticate twitter user API

In [3]:
def twi_auth():

    CONSUMER_KEY = "KIrd3dd7BSB9UT5DNa15y2DG9"
    CONSUMER_SECRET = "YDifo1C6RmIqP9aOlR9Mi83jtCm2AbXH099vCHz2ERZKgOz50s"
    ACCESS_KEY = "1058038441877753856-WpIEfvaiznbh0FTkntAdpAJ3i1lR53"
    ACCESS_SECRET = "VjUZI1jQYoHsRzovInyXL1tpyBg2NPIYID4WSX2MmCNZ8"

    auth = OAuth1(CONSUMER_KEY, CONSUMER_SECRET, ACCESS_KEY, ACCESS_SECRET)

    return auth

#### Function to get tweet count

 - Used Web Scrapping  to get total tweets from a particular twitter handle
 - Returns the tweet count

In [5]:
def twi_count(twi_handle):
    
    temp = requests.get('https://twitter.com/'+twi_handle)
    bs = BeautifulSoup(temp.text,'lxml')
    count=0
    
    try:
        tweet_box = bs.find('li',{'class':'ProfileNav-item ProfileNav-item--tweets is-active'})
        tweets= tweet_box.find('a').find('span',{'class':'ProfileNav-value'})
        count=tweets.get('data-count')

    except:
        print('Account name not found...')
        sys.exit(1)
    
    return count
        

#### Function to get the Tweets in a json response using tweepy

 - Passing the number of tweets in as the function argument so as to fetch All the tweets in real time.

In [28]:
def twitterfeed(auth, no_of_tweets, twi_handle):
   

   
   
#     auth = auth
#     api = tweepy.API(auth)
#     part_URL= \
#         "?screen_name={}&tweet_mode=extended&count={}".format(
#             twi_handle, no_of_tweets)
#     qs = requests.get('https://api.twitter.com/1.1/statuses/user_timeline.json'+ part_URL)
#     max_id = requests.get('max_id')
#     search_results = api.search(q=qs, count=no_of_tweets, max_id=max_id)
#     json_str = json.dumps( search_results )
    
   
   
    part_URL= 'https://api.twitter.com/1.1/statuses/user_timeline.json'
    URL = part_URL + \
        "?screen_name={}&tweet_mode=extended&count={}".format(
            twi_handle, no_of_tweets)

    try:
        response = requests.get(URL, auth=auth)
    except requests.exceptions.RequestException:
        print('No internet connection')
        sys.exit(1)

    data = response.json()

    return data
    

#### save_tweet_into_file accepts the JSON data and save it into file named tweets.json

In [14]:
def save_tweets_into_file(data):

    with open('tweets.json', 'w') as f:
        json.dump(data, f, indent=4)

#### main function to call the required functions in an oderly fashion to get optimum results

In [29]:
def main():

    twi_handle = "@midasIIITD"
    twi_handle2 = "midasIIITD"
    URL = 'https://api.twitter.com/1.1/statuses/user_timeline.json'

    auth = twi_auth()
    no_of_tweets = twi_count(twi_handle)
    data = twitterfeed(auth, no_of_tweets, twi_handle2)
    save_tweets_into_file(data)


if __name__ == '__main__':
    main()

# Displaying the JSON in Tabular Form

#### Function to count the Media in a Tweet

In [ ]:
def mediaCount(itr):
    
    count = 0
    
    if 'extended_entities' in itr:
        try:
            count += len(itr['extended_entities']['media'])
        except KeyError:
            pass
    
    elif 'media' in itr['entities']:
        count += len(itr['entities']['media'])
    
    # Need to check if there is media available in retweet
    if 'retweeted_status' in itr:
        count += get_all_mediaCount(itr['retweeted_status'])
    
    return count